## Análise de Vendas

In [42]:
import pandas as pd
import os

#### 1) Lendo arquivos

In [43]:
from os import listdir

lista_csv = [file for file in listdir('vendas')]

for arquivo in lista_csv:
    print(arquivo)

Devolucoes - Belo Horizonte.csv
Devolucoes - Curitiba.csv
Devolucoes - Fortaleza.csv
Devolucoes - Goiás.csv
Devolucoes - Porto Alegre.csv
Devolucoes - Recife.csv
Devolucoes - Rio de Janeiro.csv
Devolucoes - Salvador.csv
Devolucoes - São Paulo.csv
Vendas - Belo Horizonte.csv
Vendas - Curitiba.csv
Vendas - Fortaleza.csv
Vendas - Goiás.csv
Vendas - Porto Alegre.csv
Vendas - Recife.csv
Vendas - Rio de Janeiro.csv
Vendas - Salvador.csv
Vendas - São Paulo.csv


### 2) Unificando em uma única tabela .csv

In [44]:
uni_tabela_vendas = pd.DataFrame()

for arquivo in lista_csv:
    if "Vendas" in arquivo:
        tabela = pd.read_csv(f'vendas/{arquivo}')
        uni_tabela_vendas = pd.concat([uni_tabela_vendas, tabela])

uni_tabela_vendas.to_csv('uni_tabela_vendas', index= False)


In [45]:
uni_tabela_vendas = pd.read_csv('uni_tabela_vendas')

display(uni_tabela_vendas)

,Unnamed: 0,SKU,Produto,Quantidade Vendida,Primeiro Nome,Sobrenome,Data,Loja,Preco Unitario,Unnamed: 8
0,17,HL4379,Televisão,2,Carolina,Alfradique,2/25/2018,Belo Horizonte,2500,NaN
1,25,HL4379,Televisão,1,Danilo,Rubim,2/20/2018,Belo Horizonte,2500,NaN
2,27,HL1918,iPhone,5,Bernard,Pedrosa,7/7/2018,Belo Horizonte,5300,NaN
3,54,HL1918,iPhone,5,Lucas,Lemos,12/26/2018,Belo Horizonte,5300,NaN
4,67,HL8851,Notebook,5,Bernardo,Botelho,6/8/2018,Belo Horizonte,3500,NaN
...,...,...,...,...,...,...,...,...,...,...
9994,9981,HL7348,SmartWatch,4,João,Junior,4/14/2018,São Paulo,1400,NaN
9995,9984,HL1918,iPhone,4,Itai,Puntel,5/2/2018,São Paulo,5300,NaN
9996,9985,HL1918,iPhone,1,Guilherme,Vianna,3/18/2018,São Paulo,5300,NaN
9997,9991,HL7348,SmartWatch,5,Antônio,Soares,11/21/2018,São Paulo,1400,NaN


### Removendo os dados NaN da tabela

In [46]:
uni_tabela_vendas.isna().sum()

Unnamed: 0               0
SKU                      0
Produto                  0
Quantidade Vendida       0
Primeiro Nome            0
Sobrenome                0
Data                     0
Loja                     0
Preco Unitario           0
Unnamed: 8            9999
dtype: int64

In [47]:
uni_tabela_vendas.drop('Unnamed: 8', inplace=True, axis=1)

In [48]:
uni_tabela_vendas.to_csv('uni_tabela_vendas', index= False)

In [49]:
uni_tabela_vendas = pd.read_csv('uni_tabela_vendas')

display(uni_tabela_vendas)

,Unnamed: 0,SKU,Produto,Quantidade Vendida,Primeiro Nome,Sobrenome,Data,Loja,Preco Unitario
0,17,HL4379,Televisão,2,Carolina,Alfradique,2/25/2018,Belo Horizonte,2500
1,25,HL4379,Televisão,1,Danilo,Rubim,2/20/2018,Belo Horizonte,2500
2,27,HL1918,iPhone,5,Bernard,Pedrosa,7/7/2018,Belo Horizonte,5300
3,54,HL1918,iPhone,5,Lucas,Lemos,12/26/2018,Belo Horizonte,5300
4,67,HL8851,Notebook,5,Bernardo,Botelho,6/8/2018,Belo Horizonte,3500
...,...,...,...,...,...,...,...,...,...
9994,9981,HL7348,SmartWatch,4,João,Junior,4/14/2018,São Paulo,1400
9995,9984,HL1918,iPhone,4,Itai,Puntel,5/2/2018,São Paulo,5300
9996,9985,HL1918,iPhone,1,Guilherme,Vianna,3/18/2018,São Paulo,5300
9997,9991,HL7348,SmartWatch,5,Antônio,Soares,11/21/2018,São Paulo,1400
